In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import sys
import time, datetime
import json
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import (cross_val_score, train_test_split)
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.feature_selection import (RFE,RFECV)
from sklearn.ensemble import ( RandomForestClassifier , ExtraTreesClassifier )
from sklearn.metrics import ( accuracy_score, precision_score, recall_score, f1_score,roc_auc_score )
from sklearn.decomposition import PCA, IncrementalPCA, FastICA
import pickle

In [2]:
estoy_en_los_labos = True
prueba_full = True

## Dividimos la base en Train y Test (selectividad 0.2)

In [3]:
print 'Cargando ham'
if not prueba_full:
    ham_txt = json.load(open('../ham_dev_mini.json'))
elif estoy_en_los_labos:
    ham_txt = json.load(open('/media/libre/dataset_dev/ham_dev.json'))
else:
    ham_txt = json.load(open('../ham_dev.json'))
print 'Cargando spam'
if not prueba_full:
    spam_txt = json.load(open('../spam_dev_mini.json'))
elif estoy_en_los_labos:
    spam_txt = json.load(open('/media/libre/dataset_dev/spam_dev.json'))
else:
    spam_txt = json.load(open('../spam_dev.json'))

df = pd.DataFrame(ham_txt+spam_txt, columns=['text'])
df['class'] = ['ham' for _ in range(len(ham_txt))]+['spam' for _ in range(len(spam_txt))]
train, test = train_test_split(df, test_size = 0.2)
if estoy_en_los_labos:
    np.save('/media/libre/aa/train', train)
    np.save('/media/libre/aa/test', test)
else:
    np.save('train', train)
    np.save('test', test)

Cargando ham
Cargando spam


## Seleccionamos las palabras frecuentes

In [4]:
if estoy_en_los_labos:
    train = np.load('/media/libre/aa/train.npy')
else:
    train = np.load('train.npy')

print "Base Cargada"

ham = [x[0] for x in train if x[1] == 'ham']
spam = [x[0] for x in train if x[1] == 'spam']

print "Divididos Ham/Spam"

ham_counter = Counter()
spam_counter = Counter()

for s in spam:
    for w in s.split():
        spam_counter[w] += 1

for s in ham:
    for w in s.split():
        ham_counter[w] += 1

print "Palabras Contadas"

spam_words = Counter()
proportions = Counter()

for s in spam_counter:
    if s in ham_counter:
        proportions[s] = float(spam_counter[s]) / float(ham_counter[s])
    else:
        spam_words[s] = float(spam_counter[s])

print "Proporciones Calculadas"



Base Cargada
Divididos Ham/Spam
Palabras Contadas
Proporciones Calculadas


In [21]:
cant_words = 500
spam_words_list = spam_words.most_common(cant_words)
proportions_list = proportions.most_common(cant_words)

## Escribimos el archivo attributes.py

In [24]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
st1 = u"dnames = ['len',"
st2 = u""
st3 = u"dfuncs = [len,"
h=0
for t in proportions_list:
    h = h+1
    st1 += "'a" + str(h) + "',"
    st3 += "a" + str(h) + ","
    #st2 += "def a" + str(h) + "(txt): return txt.count('" + t[0].replace("'","\\'") + "')\n"
    st2 += "def a" + str(h) + "(txt): return txt.count('" + t[0] + "')\n"
    if h == cant_words:
        break
for t in spam_words_list:
    h = h+1
    st1 += "'a" + str(h) + "',"
    st3 += "a" + str(h) + ","
    #st2 += "def a" + str(h) + "(txt): return txt.count('" + t[0].replace("'","\\'") + "')\n"
    st2 += "def a" + str(h) + "(txt): return txt.count('" + t[0] + "')\n"
st1= st1[0:-1] + "]"
st3= st3[0:-1] + "]"
#print st1
#print st2
#print st3
f = open('attributes.py','w')
f.write(st1+u'\n'+st2+u'\n'+st3)
f.close()
